In [6]:
# Import the required libraries
from pyspark import SparkContext, SparkConf

# Step 1: Create a Spark Configuration and Spark Context
conf = SparkConf().setAppName("SparkBasics").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Step 2: Create an RDD from a list
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)

# Step 3: Perform operations on the RDD
# Example: Multiply each element by 2
mapped_rdd = rdd.map(lambda x: x * 2)

# Collect the result
result = mapped_rdd.collect()

# Step 4: Print the result
print("Original Data:", data)
print("Transformed Data (multiplied by 2):", result)

# Stop the SparkContext
sc.stop()

Original Data: [1, 2, 3, 4, 5]
Transformed Data (multiplied by 2): [2, 4, 6, 8, 10]


In [ ]:

from pyspark import SparkContext, SparkConf

# Step 1: Set up SparkContext
conf = SparkConf().setAppName("MinMaxTemperature").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Step 2: Sample temperature data in the form (Location, Temperature)
data = [
    ("New York", 30),
    ("Los Angeles", 25),
    ("New York", 28),
    ("Los Angeles", 22),
    ("New York", 35),
    ("Los Angeles", 20),
    ("Chicago", 15),
    ("Chicago", 10),
    ("Chicago", 12)
]

# Step 3: Create an RDD from the data
rdd = sc.parallelize(data)

# Step 4: Group data by location and calculate both min and max temperatures for each location
grouped_rdd = rdd.groupByKey()

# Find both minimum and maximum temperatures
min_max_temp_rdd = grouped_rdd.mapValues(lambda temps: (min(temps), max(temps)))

# Step 5: Collect the results and display
result = min_max_temp_rdd.collect()
print("Minimum and Maximum Temperatures by Location:")
for location, (min_temp, max_temp) in result:
    print(f"{location}: Min = {min_temp}°C, Max = {max_temp}°C")

# Stop the SparkContext
sc.stop()

Minimum and Maximum Temperatures by Location:
New York: Min = 28°C, Max = 35°C
Los Angeles: Min = 20°C, Max = 25°C
Chicago: Min = 10°C, Max = 15°C


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, count

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("WordCountDataFrame") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with text data
data = [("Hello world spark spark",), ("Hello again spark",), ("Welcome to the world of spark",)]
columns = ["line"]
df = spark.createDataFrame(data, columns)

# Step 3: Split lines into words, explode to create one word per row
words_df = df.select(explode(split(col("line"), " ")).alias("word"))

# Step 4: Count occurrences of each word
word_count_df = words_df.groupBy("word").agg(count("word").alias("count"))

# Step 5: Show the result
word_count_df.show()

# Stop the SparkSession
spark.stop()

+-------+-----+
|   word|count|
+-------+-----+
|  Hello|    2|
|  spark|    4|
|  world|    2|
|  again|    1|
|    the|    1|
|     of|    1|
|Welcome|    1|
|     to|    1|
+-------+-----+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, col

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("SQLCommandsOnDataFrame") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with sample data
data = [
    ("Alice", "Electronics", 200),
    ("Bob", "Electronics", 150),
    ("Alice", "Clothing", 100),
    ("Bob", "Clothing", 50),
    ("Alice", "Electronics", 300)
]
columns = ["Name", "Category", "Amount"]
df = spark.createDataFrame(data, columns)

# Step 3: Execute SQL-style functions
# Calculate total and average spending by category
result_df = df.groupBy("Category") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        avg("Amount").alias("AverageSpent")
    )

# Step 4: Show the result
result_df.show()

# Stop the SparkSession
spark.stop()

+-----------+----------+------------------+
|   Category|TotalSpent|      AverageSpent|
+-----------+----------+------------------+
|Electronics|       650|216.66666666666666|
|   Clothing|       150|              75.0|
+-----------+----------+------------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("TotalSpentByCustomer") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with sample data
data = [
    ("Alice", 200),
    ("Bob", 150),
    ("Alice", 100),
    ("Bob", 50),
    ("Alice", 300)
]
columns = ["Customer", "Amount"]
df = spark.createDataFrame(data, columns)

# Step 3: Group by Customer and calculate the total amount spent
result_df = df.groupBy("Customer").agg(sum("Amount").alias("TotalSpent"))

# Step 4: Show the result
result_df.show()

# Stop the SparkSession
spark.stop()

+--------+----------+
|Customer|TotalSpent|
+--------+----------+
|     Bob|       200|
|   Alice|       600|
+--------+----------+



In [ ]:
from pyspark.sql import SparkSession

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("BroadcastVariablesExample") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with movie IDs and ratings
movie_data = [
    (1, 4.5),
    (2, 3.0),
    (1, 5.0),
    (3, 4.0)
]
columns = ["MovieID", "Rating"]
ratings_df = spark.createDataFrame(movie_data, columns)

# Step 3: Create a dictionary with movie names
movie_names = {
    1: "Inception",
    2: "Interstellar",
    3: "The Dark Knight"
}

# Step 4: Broadcast the movie names dictionary
movie_names_broadcast = spark.sparkContext.broadcast(movie_names)

# Step 5: Map MovieID to Movie Name using the broadcast variable
def map_movie_name(movie_id):
    return movie_names_broadcast.value.get(movie_id, "Unknown")

# Register the function as a UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

map_movie_name_udf = udf(map_movie_name, StringType())

# Add a new column with movie names
result_df = ratings_df.withColumn("MovieName", map_movie_name_udf(ratings_df.MovieID))

# Step 6: Show the result
result_df.show()

# Stop the SparkSession
spark.stop()

+-------+------+---------------+
|MovieID|Rating|      MovieName|
+-------+------+---------------+
|      1|   4.5|      Inception|
|      2|   3.0|   Interstellar|
|      1|   5.0|      Inception|
|      3|   4.0|The Dark Knight|
+-------+------+---------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import IntegerType, FloatType

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("MovieRecommendations") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with sample user-movie ratings
data = [
    (0, 1, 4.0),  # User 0 rated Movie 1 with 4.0
    (0, 2, 2.0),  # User 0 rated Movie 2 with 2.0
    (1, 2, 5.0),  # User 1 rated Movie 2 with 5.0
    (1, 3, 3.0),  # User 1 rated Movie 3 with 3.0
    (2, 1, 1.0),  # User 2 rated Movie 1 with 1.0
    (2, 3, 4.0)   # User 2 rated Movie 3 with 4.0
]
columns = ["UserID", "MovieID", "Rating"]
ratings_df = spark.createDataFrame(data, columns)

# Step 3: Create an ALS model
als = ALS(
    maxIter=5,  # Number of iterations
    regParam=0.01,  # Regularization parameter
    userCol="UserID",  # Column for users
    itemCol="MovieID",  # Column for items
    ratingCol="Rating",  # Column for ratings
    coldStartStrategy="drop"  # Drop rows with null predictions
)

# Step 4: Train the ALS model
model = als.fit(ratings_df)

# Step 5: Generate top 2 movie recommendations for each user
user_recommendations = model.recommendForAllUsers(2)

# Step 6: Show the recommendations
user_recommendations.show(truncate=False)

# Stop the SparkSession
spark.stop()

+------+-------------------------------+
|UserID|recommendations                |
+------+-------------------------------+
|0     |[{1, 3.994191}, {2, 2.0000634}]|
|1     |[{2, 4.996521}, {3, 3.0001428}]|
|2     |[{3, 3.9961724}, {2, 3.443232}]|
+------+-------------------------------+



In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("MostViewedURL") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with sample log data
data = [
    (1, "http://example.com/home"),
    (2, "http://example.com/about"),
    (3, "http://example.com/home"),
    (4, "http://example.com/contact"),
    (5, "http://example.com/home"),
    (6, "http://example.com/about"),
    (7, "http://example.com/contact"),
    (8, "http://example.com/home")
]
columns = ["UserID", "URL"]
logs_df = spark.createDataFrame(data, columns)

# Step 3: Count the number of views for each URL
url_counts_df = logs_df.groupBy("URL").agg(count("URL").alias("ViewCount"))

# Step 4: Sort the URLs by the number of views in descending order
sorted_url_counts_df = url_counts_df.orderBy(desc("ViewCount"))

# Step 5: Show the most viewed URLs
sorted_url_counts_df.show()

# Stop the SparkSession
spark.stop()

+--------------------+---------+
|                 URL|ViewCount|
+--------------------+---------+
|http://example.co...|        4|
|http://example.co...|        2|
|http://example.co...|        2|
+--------------------+---------+

